## ✅ Notebook Goal:
 Explore and evaluate the structure, density, and effectiveness of the SQLite schema design


In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# --- Connect to SQLite ---
conn = sqlite3.connect("../../sql/openfda_base_updated.db")
cursor = conn.cursor()

In [2]:
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

def get_columns(table_name):
    query = f"PRAGMA table_info({table_name})"
    columns = conn.execute(query).fetchall()
    return [col[1] for col in columns]

for table in tables:
    print(f"Table: {table[0]} has {len(get_columns(table[0]))} columns")

Table: report has 28 columns
Table: reaction has 5 columns
Table: report_duplicate has 4 columns
Table: patient_optional has 6 columns
Table: drug has 13 columns
Table: drug_openfda has 20 columns
Table: drug_activesubstance has 3 columns
Table: drug_optional has 15 columns
Table: summary has 4 columns
Table: primarysource_literature_reference has 3 columns


In [3]:
get_columns(tables[0][0])

['safetyreportid',
 'safetyreportversion',
 'receivedateformat',
 'receivedate',
 'receiptdateformat',
 'receiptdate',
 'transmissiondateformat',
 'transmissiondate',
 'companynumb',
 'reporttype',
 'fulfillexpeditecriteria',
 'serious',
 'seriousnessdeath',
 'seriousnesslifethreatening',
 'seriousnesshospitalization',
 'seriousnessdisabling',
 'seriousnesscongenitalanomali',
 'seriousnessother',
 'primarysourcecountry',
 'sendertype',
 'senderorganization',
 'receivertype',
 'receiverorganization',
 'primarysource_qualification',
 'primarysource_reportercountry',
 'authoritynumb',
 'occurcountry',
 'duplicate']

## Parent fields analysis

In [24]:
# --- 1. Drugs per report ---
query = """
SELECT COUNT(*) AS num_reports, drug_count
FROM (
  SELECT safetyreportid, COUNT(*) AS drug_count
  FROM drug
  GROUP BY safetyreportid
) AS report_drug_counts
GROUP BY drug_count
ORDER BY drug_count;
"""

cursor.execute(query)
results = cursor.fetchall()
# check type of results
print(f"Type of results: {type(results)}")

print(f"Drugs per report: {len(results)}")

for i in range(10):
    print(f"Drugs per report {i}: {results[i]}")



Type of results: <class 'list'>
Drugs per report: 166
Drugs per report 0: (13270, 1)
Drugs per report 1: (5927, 2)
Drugs per report 2: (3374, 3)
Drugs per report 3: (2705, 4)
Drugs per report 4: (1867, 5)
Drugs per report 5: (1334, 6)
Drugs per report 6: (952, 7)
Drugs per report 7: (1048, 8)
Drugs per report 8: (678, 9)
Drugs per report 9: (759, 10)


In [49]:
# --- Reactions per report ---
query = """
SELECT COUNT(*) AS num_reports, reaction_count
FROM (
  SELECT safetyreportid, COUNT(*) AS reaction_count
  FROM reaction
  GROUP BY safetyreportid
) AS report_reaction_counts
GROUP BY reaction_count
ORDER BY reaction_count;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"Type of results: {type(results)}")
print(f"Reactions per report: {len(results)}")

for i in range(min(10, len(results))):
    print(f"Reactions per report {i+1}: {results[i]}")

Type of results: <class 'list'>
Reactions per report: 89
Reactions per report 1: (12840, 1)
Reactions per report 2: (8471, 2)
Reactions per report 3: (4784, 3)
Reactions per report 4: (2751, 4)
Reactions per report 5: (1836, 5)
Reactions per report 6: (1207, 6)
Reactions per report 7: (807, 7)
Reactions per report 8: (633, 8)
Reactions per report 9: (481, 9)
Reactions per report 10: (385, 10)


In [43]:
# --- Reports per year (based on receivedate) ---
query = """
SELECT SUBSTR(receivedate, 1, 4) AS year, COUNT(*) AS num_reports
FROM report
WHERE LENGTH(receivedate) >= 4
GROUP BY year
ORDER BY year;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"Reports by year: {len(results)} years")

for year, count in results:
    print(f"{year}: {count} reports")

Reports by year: 16 years
2007: 2 reports
2008: 2 reports
2010: 3 reports
2012: 2 reports
2013: 1 reports
2014: 10 reports
2015: 24 reports
2016: 55 reports
2017: 91 reports
2018: 110 reports
2019: 186 reports
2020: 298 reports
2021: 580 reports
2022: 1521 reports
2023: 7257 reports
2024: 25858 reports


In [45]:
# --- Distribution of report_duplicate entries per report ---
query = """
SELECT COUNT(*) AS num_duplicates, safetyreportid
FROM report_duplicate
GROUP BY safetyreportid
ORDER BY num_duplicates DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"Number of reports with duplicates: {len(results)}")

# Distribution summary

# distribution = Counter(count for count, _ in results)
# for dup_count in sorted(distribution.keys()):
#     print(f"{dup_count} duplicates: {distribution[dup_count]} reports")

Number of reports with duplicates: 5581


In [46]:
# --- Distribution of drug_openfda entries per drug ---
query = """
SELECT COUNT(*) AS num_openfda, drug_id
FROM drug_openfda
GROUP BY drug_id
ORDER BY num_openfda DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"Number of drugs with openfda info: {len(results)}")

distribution = Counter(count for count, _ in results)
for entry_count in sorted(distribution.keys()):
    print(f"{entry_count} openfda records: {distribution[entry_count]} drugs")

Number of drugs with openfda info: 188847
1 openfda records: 188847 drugs


In [47]:
# --- Distribution of drug_activesubstance entries per drug ---
query = """
SELECT COUNT(*) AS num_substances, drug_id
FROM drug_activesubstance
GROUP BY drug_id
ORDER BY num_substances DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"Number of drugs with activesubstance info: {len(results)}")

distribution = Counter(count for count, _ in results)
for entry_count in sorted(distribution.keys()):
    print(f"{entry_count} substances: {distribution[entry_count]} drugs")

Number of drugs with activesubstance info: 185151
1 substances: 185151 drugs


## Nested list completeness

In [48]:
query = """
SELECT COUNT(DISTINCT safetyreportid) AS with_reactions
FROM reaction;
"""
cursor.execute(query)
reaction_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM report;")
report_total = cursor.fetchone()[0]

print(f"Reports with ≥1 reaction: {reaction_count} / {report_total} ({100 * reaction_count / report_total:.1f}%)")


Reports with ≥1 reaction: 36000 / 36000 (100.0%)


In [50]:
query = """
SELECT COUNT(DISTINCT safetyreportid) AS with_duplicates
FROM report_duplicate;
"""
cursor.execute(query)
duplicate_count = cursor.fetchone()[0]

print(f"Reports with ≥1 duplicate: {duplicate_count} / {report_total} ({100 * duplicate_count / report_total:.1f}%)")


Reports with ≥1 duplicate: 5581 / 36000 (15.5%)


In [51]:
query = """
SELECT COUNT(DISTINCT safetyreportid) AS with_drugs
FROM drug;
"""
cursor.execute(query)
drug_count = cursor.fetchone()[0]

print(f"Reports with ≥1 drug: {drug_count} / {report_total} ({100 * drug_count / report_total:.1f}%)")


Reports with ≥1 drug: 36000 / 36000 (100.0%)


In [52]:
query = """
SELECT COUNT(DISTINCT drug_id) AS with_openfda
FROM drug_openfda;
"""
cursor.execute(query)
openfda_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM drug;")
drug_total = cursor.fetchone()[0]

print(f"Drugs with ≥1 openfda: {openfda_count} / {drug_total} ({100 * openfda_count / drug_total:.1f}%)")


Drugs with ≥1 openfda: 188847 / 188847 (100.0%)


In [53]:
query = """
SELECT COUNT(DISTINCT drug_id) AS with_openfda
FROM drug_openfda;
"""
cursor.execute(query)
openfda_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM drug;")
drug_total = cursor.fetchone()[0]

print(f"Drugs with ≥1 openfda: {openfda_count} / {drug_total} ({100 * openfda_count / drug_total:.1f}%)")


Drugs with ≥1 openfda: 188847 / 188847 (100.0%)


## Date - Date Format

In [56]:
def summarize_date_and_format_counts(table, date_column, format_column):
    print(f"📌 Field: {date_column}")
    
    # Count grouped by format where date IS NOT NULL
    query_date_present = f"""
    SELECT {format_column} AS format_code, COUNT(*) AS date_present_count
    FROM {table}
    WHERE {date_column} IS NOT NULL
    GROUP BY {format_column}
    """

    # Count grouped by format regardless of date
    query_format_total = f"""
    SELECT {format_column} AS format_code, COUNT(*) AS total_format_entries
    FROM {table}
    GROUP BY {format_column}
    """

    df_dates = pd.read_sql_query(query_date_present, conn)
    df_format = pd.read_sql_query(query_format_total, conn)

    df = pd.merge(df_format, df_dates, on="format_code", how="outer").fillna(0)
    df = df.astype({"total_format_entries": int, "date_present_count": int})
    df = df.sort_values("format_code")

    print(df.rename(columns={
        "format_code": "Format",
        "total_format_entries": "# Format Entries",
        "date_present_count": "# Non-null Dates"
    }).to_string(index=False))
    print()

In [57]:
# Report table
summarize_date_and_format_counts("report", "receivedate", "receivedateformat")
summarize_date_and_format_counts("report", "receiptdate", "receiptdateformat")
summarize_date_and_format_counts("report", "transmissiondate", "transmissiondateformat")

# Drug table
summarize_date_and_format_counts("drug", "drugstartdate", "drugstartdateformat")
summarize_date_and_format_counts("drug", "drugenddate", "drugenddateformat")

📌 Field: receivedate
 Format  # Format Entries  # Non-null Dates
    102             36000             36000

📌 Field: receiptdate
 Format  # Format Entries  # Non-null Dates
    102             36000             36000

📌 Field: transmissiondate
 Format  # Format Entries  # Non-null Dates
    102             36000             36000

📌 Field: drugstartdate
 Format  # Format Entries  # Non-null Dates
    0.0            135011                 0
  102.0             40224             40224
  602.0              4551              4551
  610.0              9061              9061

📌 Field: drugenddate
 Format  # Format Entries  # Non-null Dates
    0.0            165696                 0
  102.0             19162             19162
  602.0              1289              1289
  610.0              2700              2700



In [60]:
def validate_drug_date_padding(date_column, format_column):
    print(f"🔍 Validating {date_column} consistency with {format_column}\n")

    # Check for format 602: year only, should end with '0101'
    query_602 = f"""
    SELECT {date_column}
    FROM drug
    WHERE {format_column} = 602
      AND SUBSTR({date_column}, -5) != '01-01'
    """
    df_602 = pd.read_sql_query(query_602, conn)
    print(f"Format 602: {len(df_602)} incorrectly padded (expected '0101' ending)")

    # Check for format 610: year+month, should end with '01'
    query_610 = f"""
    SELECT {date_column}
    FROM drug
    WHERE {format_column} = 610
      AND SUBSTR({date_column}, -2) != '01'
    """
    df_610 = pd.read_sql_query(query_610, conn)
    print(f"Format 610: {len(df_610)} incorrectly padded (expected '01' ending)")

    # Check for format 102: full date, should be exactly 8 digits
    query_102 = f"""
    SELECT {date_column}
    FROM drug
    WHERE {format_column} = 102
      AND LENGTH({date_column}) != 10
    """
    df_102 = pd.read_sql_query(query_102, conn)
    print(f"Format 102: {len(df_102)} with unexpected length (should be 8 digits)")

    print()

# Run for both date fields in drug table
validate_drug_date_padding("drugstartdate", "drugstartdateformat")
validate_drug_date_padding("drugenddate", "drugenddateformat")


🔍 Validating drugstartdate consistency with drugstartdateformat

Format 602: 0 incorrectly padded (expected '0101' ending)
Format 610: 0 incorrectly padded (expected '01' ending)
Format 102: 0 with unexpected length (should be 8 digits)

🔍 Validating drugenddate consistency with drugenddateformat

Format 602: 0 incorrectly padded (expected '0101' ending)
Format 610: 0 incorrectly padded (expected '01' ending)
Format 102: 0 with unexpected length (should be 8 digits)



## Categorical fields calidation

In [5]:
def validate_categorical_field(table, column, expected_codes=None, description=None):
    print(f"🔎 Checking values in {table}.{column}")
    if description:
        print(f"   → {description}")
    
    query = f"""
    SELECT {column} AS code, COUNT(*) AS count
    FROM {table}
    GROUP BY {column}
    ORDER BY code;
    """
    df = pd.read_sql_query(query, conn)
    print(df.to_string(index=False))
    
    if expected_codes is not None:
        found_codes = set(df["code"].dropna())
        expected_codes = set(expected_codes)
        unexpected = found_codes - expected_codes
        missing = expected_codes - found_codes

        if unexpected:
            print(f"❌ Unexpected codes found: {sorted(unexpected)}")
        else:
            print("✅ No unexpected codes.")

        if missing:
            print(f"⚠️ Expected codes missing: {sorted(missing)}")
        else:
            print("✅ All expected codes present.")
    
    print()


In [13]:
categorical_fields = [
    # {
    #     "table": "patient_optional",
    #     "column": "patientsex",
    #     "expected": [0, 1, 2],
    #     "description": "0 = Unknown, 1 = Male, 2 = Female"
    # },
    {
        "table": "patient_optional",
        "column": "patientagegroup",
        "expected": list(range(1, 10)),  # placeholder; adjust based on MedDRA
        "description": "1–9 = Standard age groups (check codes)"
    },
    # {
    #     "table": "reaction",
    #     "column": "reactionoutcome",
    #     "expected": [1, 2, 3, 4, 5, 6],
    #     "description": "MedDRA outcome codes"
    # },
    # {
    #     "table": "report",
    #     "column": "seriousnessdeath",
    #     "expected": [1, 2],
    #     "description": "1 = Yes, 2 = No"
    # },
    # {
    #     "table": "report",
    #     "column": "seriousnesshospitalization",
    #     "expected": [1, 2],
    #     "description": "1 = Yes, 2 = No"
    # },
    # {
    #     "table": "report",
    #     "column": "seriousnesslifethreatening",
    #     "expected": [1, 2],
    #     "description": "1 = Yes, 2 = No"
    # },
    # {
    #     "table": "report",
    #     "column": "reporttype",
    #     "expected": [1, 2, 3, 4, 5],  # placeholder; adjust if needed
    #     "description": "Type of report (check OpenFDA spec)"
    # },
    # {
    #     "table": "drug",
    #     "column": "drugcharacterization",
    #     "expected": [1, 2, 3],
    #     "description": "1 = Suspect, 2 = Concomitant, 3 = Interacting"
    # },
    # {
    #     "table": "drug_optional",
    #     "column": "drugintervaldosagedefinition",
    #     "expected": [802, 803, 804],  # placeholder; update based on data
    #     "description": "Interval dosage definitions (802 = Daily, etc.)"
    # },
    # {
    #     "table": "report",
    #     "column": "safetyreportversion",
    #     "expected": [1, 2, 3],  # placeholder; adjust based on data
    #     "description": "Version of the safety report (check OpenFDA spec)"
    # }
]

categorical_fields = [
    {
        "table": "patient_optional",
        "column": "patientagegroup",
        "expected": list(range(1, 10)),  # placeholder; adjust based on MedDRA
        "description": "1–9 = Standard age groups (check codes)"
    },
    {
        "table": "patient_optional",
        "column": "patientonsetageunit",
        "expected": [800, 801, 802, 803, 804, 805],
        "description": "801 = Years, 802 = Months, 803 = Days, 804 = Hours, 805 = Minutes, 806 = Seconds"
    }
]


for field in categorical_fields:
    validate_categorical_field(
        table=field["table"],
        column=field["column"],
        expected_codes=field["expected"],
        description=field["description"]
    )


🔎 Checking values in patient_optional.patientagegroup
   → 1–9 = Standard age groups (check codes)
 code  count
  NaN  23805
  1.0     47
  2.0     50
  3.0    392
  4.0    390
  5.0   6867
  6.0   4449
✅ No unexpected codes.
⚠️ Expected codes missing: [7, 8, 9]

🔎 Checking values in patient_optional.patientonsetageunit
   → 801 = Years, 802 = Months, 803 = Days, 804 = Hours, 805 = Minutes, 806 = Seconds
 code  count
  NaN  12456
800.0    338
801.0  22990
802.0    112
803.0      8
804.0     94
805.0      2
✅ No unexpected codes.
✅ All expected codes present.



## Field sparsity

In [8]:
# Prepare a collector DataFrame
all_nulls = []

def summarize_null_ratios(table_name, limit=28):
    print(f"📊 NULL summary for table: {table_name}")
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    null_ratios = df.isnull().mean().sort_values(ascending=False) * 100
    null_ratios = null_ratios[null_ratios > 0].round(2)

    display(null_ratios.head(limit).rename("NULL %"))
    
    # Add to master summary
    for col, pct in null_ratios.items():
        all_nulls.append({
            "table": table_name,
            "column": col,
            "null_pct": pct
        })
    print()

# Loop over all key tables
tables_to_check = [
    "report",
    "reaction",
    "drug",
    "drug_optional",
    "drug_openfda",
    "drug_activesubstance",
    "patient_optional",
    "report_duplicate",
    "summary",
    "primarysource_literature_reference"
]

for table in tables_to_check:
    summarize_null_ratios(table)

# Combine into DataFrame
null_summary_df = pd.DataFrame(all_nulls)

📊 NULL summary for table: report


authoritynumb                   89.46
duplicate                       67.46
companynumb                     10.88
occurcountry                     7.06
seriousnesscongenitalanomali     3.59
seriousnessdisabling             3.56
seriousnessdeath                 3.27
seriousnesslifethreatening       3.20
seriousnesshospitalization       2.32
seriousnessother                 2.18
primarysource_qualification      0.61
primarysourcecountry             0.40
reporttype                       0.07
Name: NULL %, dtype: float64


📊 NULL summary for table: reaction


reactionoutcome    3.07
Name: NULL %, dtype: float64


📊 NULL summary for table: drug


...

duplicatesource    4.85
duplicatenumb      0.01
Name: NULL %, dtype: float64


📊 NULL summary for table: summary


Series([], Name: NULL %, dtype: float64)


📊 NULL summary for table: primarysource_literature_reference


Series([], Name: NULL %, dtype: float64)

In [91]:
null_summary_df.to_csv("../../reports/evaluation_results/sql_null_summary.csv", index=False)
print("✅ Exported sparse fields to: reports/evaluation_results/sql_null_summary.csv")

✅ Exported sparse fields to: reports/evaluation_results/sql_null_summary.csv


In [68]:
from matplotlib import pyplot as plt

# Collect nulls for all tables
null_summary = []

for table in tables_to_check:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    null_ratios = df.isnull().mean() * 100
    for col, pct in null_ratios.items():
        if pct > 0:
            null_summary.append({
                "table": table,
                "column": col,
                "null_pct": round(pct, 2)
            })

null_df = pd.DataFrame(null_summary)


In [90]:
# # Group NULLs by table
# nulls_by_table = null_df.groupby("table")

# for table_name, group in nulls_by_table:
#     group_sorted = group.sort_values("null_pct", ascending=True)

#     plt.figure(figsize=(8, 0.4 * len(group_sorted) + 1))
#     plt.barh(
#         y=group_sorted["column"],
#         width=group_sorted["null_pct"]
#     )
#     plt.xlabel("% NULL")
#     plt.title(f"{table_name}: NULL Percentage per Column")
#     plt.grid(axis="x", linestyle="--", alpha=0.5)
#     plt.tight_layout()
#     plt.show()

In [14]:
query = """
SELECT COUNT(*)
FROM report
WHERE duplicate IS NOT NULL
AND safetyreportid NOT IN (
  SELECT safetyreportid FROM report_duplicate
);
"""
cursor.execute(query)
missing_duplicates = cursor.fetchone()[0]
print(f"Reports with duplicate info but no report_duplicate entry: {missing_duplicates}")

Reports with duplicate info but no report_duplicate entry: 6134


## IDs - drugs


In [96]:
query = """
SELECT COUNT(DISTINCT medicinalproduct) AS num_unique_drug_names
FROM drug;
"""

cursor.execute(query)
result = cursor.fetchone()[0]
print(f"Total number of distinct drug names: {result}")


Total number of distinct drug names: 8223


In [95]:
query = """
SELECT d.medicinalproduct, COUNT(DISTINCT a.activesubstancename) AS unique_substance_count
FROM drug d
JOIN drug_activesubstance a ON d.id = a.drug_id
GROUP BY d.medicinalproduct
ORDER BY unique_substance_count DESC;
"""

df = pd.read_sql_query(query, conn)
print(df.shape)
print("Top 20 drugs by number of unique active substances:")
display(df)


(5722, 2)
Top 20 drugs by number of unique active substances:


,medicinalproduct,unique_substance_count
0,SARCLISA,2
1,RUCONEST,2
2,RASAGILINE,2
3,PRAMIPEXOLE,2
4,OXYCODONE,2
...,...,...
5717,".ALPHA.-TOCOPHEROL ACETATE, D-\EVENING PRIMROS...",1
5718,".ALPHA.-TOCOPHEROL ACETATE, D-",1
5719,.ALPHA.-TOCOPHEROL ACETATE,1
5720,.ALPHA.-TOCOPHEROL,1


In [7]:
# distinct activesubstance names
query = """
SELECT COUNT(DISTINCT activesubstancename) AS num_unique_substances
FROM drug_activesubstance;
"""
cursor.execute(query)
result = cursor.fetchone()[0]
print(f"Total number of distinct active substances: {result}")


Total number of distinct active substances: 3750


In [100]:
query = """
SELECT d.medicinalproduct, COUNT(DISTINCT a.activesubstancename) AS unique_substance_count
FROM drug d
JOIN drug_activesubstance a ON d.id = a.drug_id
GROUP BY d.medicinalproduct
ORDER BY unique_substance_count DESC;
"""

df = pd.read_sql_query(query, conn)
print(df.shape)
print("Top 20 drugs by number of unique active substances:")
display(df)

(5722, 2)
Top 20 drugs by number of unique active substances:


,medicinalproduct,unique_substance_count
0,SARCLISA,2
1,RUCONEST,2
2,RASAGILINE,2
3,PRAMIPEXOLE,2
4,OXYCODONE,2
...,...,...
5717,".ALPHA.-TOCOPHEROL ACETATE, D-\EVENING PRIMROS...",1
5718,".ALPHA.-TOCOPHEROL ACETATE, D-",1
5719,.ALPHA.-TOCOPHEROL ACETATE,1
5720,.ALPHA.-TOCOPHEROL,1


In [104]:
query = """
SELECT d.medicinalproduct,
       COUNT(DISTINCT a.activesubstancename) AS substance_count,
       GROUP_CONCAT(DISTINCT a.activesubstancename) AS substances
FROM drug d
JOIN drug_activesubstance a ON d.id = a.drug_id
WHERE d.medicinalproduct IS NOT NULL AND a.activesubstancename IS NOT NULL
GROUP BY d.medicinalproduct
ORDER BY substance_count DESC
LIMIT 50;
"""

df = pd.read_sql_query(query, conn)
df["substances"] = df["substances"].str.replace(",", ", ")
display(df)


,medicinalproduct,substance_count,substances
0,SARCLISA,2,"ISATUXIMAB\ISATUXIMAB-IRFC, ISATUXIMAB-IRFC"
1,RUCONEST,2,"CONESTAT ALFA, HUMAN C1-ESTERASE INHIBITOR"
2,RASAGILINE,2,"RASAGILINE\RASAGILINE MESYLATE, RASAGILINE"
3,PRAMIPEXOLE,2,"PRAMIPEXOLE\PRAMIPEXOLE DIHYDROCHLORIDE, PRAMI..."
4,OXYCODONE,2,"OXYCODONE\OXYCODONE HYDROCHLORIDE, OXYCODONE"
5,OXYBUTYNIN,2,"OXYBUTYNIN\OXYBUTYNIN CHLORIDE, OXYBUTYNIN"
6,OCTREOTIDE,2,"OCTREOTIDE\OCTREOTIDE ACETATE, OCTREOTIDE"
7,NOREPINEPHRINE,2,"NOREPINEPHRINE, NOREPINEPHRINE\NOREPINEPHRINE ..."
8,LEVOTHYROXINE,2,"LEVOTHYROXINE\LEVOTHYROXINE SODIUM, LEVOTHYROXINE"
9,LEUPROLIDE ACETATE,2,"LEUPROLIDE ACETATE, ISOPROPYL ALCOHOL\LEUPROLI..."


In [2]:
# Cleanup
conn.close()
